# Chapter 2: Routing

## Hands-On Code Example (Google ADK)

> Adapted and modified from https://docs.google.com/document/d/1RZ5-2fykDQKOBx01pwfKkDe0GCs5ydca7xW9Q4wqS_M/edit?tab=t.0
> 
> Mi  8 Okt 2025 16:20:48 BST

The following code example demonstrates the establishment of a hierarchical agent structure within the Google ADK through the creation of a parent-child relationship. The code defines two types of agents: LlmAgent and a custom TaskExecutor agent derived from BaseAgent. The TaskExecutor is designed for specific, non-LLM tasks and in this example, it simply yields a "Task finished successfully" event. An LlmAgent named greeter is initialized with a specified model and instruction to act as a friendly greeter. The custom TaskExecutor is instantiated as task_doer. A parent LlmAgent called coordinator is created, also with a model and instructions. The coordinator's instructions guide it to delegate greetings to the greeter and task execution to the task_doer. The greeter and task_doer are added as sub-agents to the coordinator, establishing a parent-child relationship. The code then asserts that this relationship is correctly set up. Finally, it prints a message indicating that the agent hierarchy has been successfully created.

In [1]:
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event
from typing import AsyncGenerator

In [2]:
# Correctly implement a custom agent by extending BaseAgent
class TaskExecutor(BaseAgent):
   """A specialized agent with custom, non-LLM behavior."""
   name: str = "TaskExecutor"
   description: str = "Executes a predefined task."

   async def _run_async_impl(self, context: InvocationContext) -> AsyncGenerator[Event, None]:
       """Custom implementation logic for the task."""
       # This is where your custom logic would go.
       # For this example, we'll just yield a simple event.
       yield Event(author=self.name, content="Task finished successfully.")

**Note**: 

This code defines an asynchronous method that implements custom behavior for a Google ADK agent. Let`s break it down:

**Method Signature** 

```python
async def _run_async_impl(self, context: InvocationContext) -> AsyncGenerator[Event, None]:
``` 

- `async def`: This is an asynchronous function that can be paused and resumed, allowing other code to run while waiting for operations to complete

- `_run_async_impl`: This is a protected method (indicated by the leading underscore) that serves as the core implementation for the agent's behavior

- `self`: Reference to the current agent instance

- `context: InvocationContext`: Contains information about the current execution context, such as session state, user input, and other runtime data

- `-> AsyncGenerator[Event, None]`: The return type annotation indicating this function yields `Event` objects asynchronously

**What AsyncGenerator Means**

An `AsyncGenerator` is like a regular generator but for asynchronous code. Instead of returning all values at once, it yields values one at a time, and each yield can be awaited. This is perfect for agent interactions where you might want to:

- Send multiple messages over time

- Process data in chunks

- Provide real-time feedback

**The Implementation**

```python
yield Event(author=self.name, content="Task finished successfully.")
``` 

- `yield`: Instead of `return`, this produces a value that can be consumed by the caller, but the function continues to exist and can yield more values

- `Event(...)`: Creates an event object that represents something happening in the agent system

- `author=self.name`: Sets the event's author to this agent's name

- `content="Task finished successfully."`: The actual message or data being communicated

**Why This Pattern?**

This design allows agents to:

1. **Stream responses**: Send multiple events over time rather than one big response

2. **Provide updates**: Give progress reports during long-running tasks

3. **Chain interactions**: Other agents can react to each event as it's yielded

4. **Handle interruptions**: The async nature allows the system to handle cancellations or timeouts gracefully


**Example Usage Context**

In the broader context of your Google ADK agent hierarchy, when the `TaskExecutor` agent is invoked, this method will:

1. Execute whatever custom logic you implement

2. Yield an event indicating successful completion

3. Allow the parent `Coordinator` agent to receive and potentially act on this event

This is a simple example, but in a real implementation, you might yield multiple events showing progress, results, or different types of notifications.

In [3]:
# Define individual agents with proper initialization
# LlmAgent requires a model to be specified.
greeter = LlmAgent(
   name="Greeter",
   model="gemini-2.0-flash-exp",
   instruction="You are a friendly greeter."
)
task_doer = TaskExecutor() # Instantiate our concrete custom agent

In [4]:
# Create a parent agent and assign its sub-agents
# The parent agent's description and instructions should guide its delegation logic.
coordinator = LlmAgent(
   name="Coordinator",
   model="gemini-2.0-flash-exp",
   description="A coordinator that can greet users and execute tasks.",
   instruction="When asked to greet, delegate to the Greeter. When asked to perform a task, delegate to the TaskExecutor.",
   sub_agents=[
       greeter,
       task_doer
   ]
)

In [5]:
# The ADK framework automatically establishes the parent-child relationships.
# These assertions will pass if checked after initialization.
assert greeter.parent_agent == coordinator
assert task_doer.parent_agent == coordinator

print("Agent hierarchy created successfully.")

Agent hierarchy created successfully.


This code excerpt illustrates the employment of the LoopAgent within the Google ADK framework to establish iterative workflows. The code defines two agents: ConditionChecker and ProcessingStep. ConditionChecker is a custom agent that checks a "status" value in the session state. If the "status" is "completed", ConditionChecker escalates an event to stop the loop. Otherwise, it yields an event to continue the loop. ProcessingStep is an LlmAgent using the "gemini-2.0-flash-exp" model. Its instruction is to perform a task and set the session "status" to "completed" if it's the final step. A LoopAgent named StatusPoller is created. StatusPoller is configured with max_iterations=10. StatusPoller includes both ProcessingStep and an instance of ConditionChecker as sub-agents. The LoopAgent will execute the sub-agents sequentially for up to 10 iterations, stopping if ConditionChecker finds the status is "completed".